In [ ]:
import os, sys
from tqdm import trange

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy


import torch
from torch import nn
from torch.autograd.functional import jacobian


source = "/home/loek/projects/rnn/source"
sys.path.append(source)


from data import seq_data, grid_data
from preprocessing import OneHot
from compilation import Compiler, Tracker, ScalarTracker, ActivationTracker
from data_analysis.automata import to_automaton_history
from data_analysis.visualization.animation import SliderAnimation
from data_analysis.visualization.activations import ActivationsAnimation
from data_analysis.visualization.automata import AutomatonAnimation
from data_analysis.visualization.epochs import EpochAnimation

from model import Model

import cProfile
import pstats


is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

device = torch.device("cpu")


In [ ]:
## Generate data
symbols = [0, 1]
encoding = OneHot(symbols)
xor_problem = lambda seq: np.sum(seq) % 2
par_3_problem = lambda seq: int(np.sum(seq) % 3 == 2)

problem = xor_problem

N = 15

train_datasets = [
    seq_data(device, problem, encoding, seq_len=1, n_datapoints=N),
    seq_data(device, problem, encoding, seq_len=2, n_datapoints=N),
    seq_data(device, problem, encoding, seq_len=3, n_datapoints=N),
    seq_data(device, problem, encoding, seq_len=4, n_datapoints=N),
    # seq_data(device, problem, encoding, seq_len=5, n_datapoints=N),
    # seq_data(device, problem, encoding, seq_len=6, n_datapoints=N),
    # seq_data(device, problem, encoding, seq_len=7, n_datapoints=N),
    # seq_data(device, problem, encoding, seq_len=8, n_datapoints=N),
    # seq_data(device, problem, encoding, seq_len=9, n_datapoints=N),
]

val_dataset = [seq_data(device, problem, encoding, n_datapoints=50, seq_len=10)]
tracked_datasets = val_dataset + train_datasets


grid = grid_data(device, dim=2, output_dim=2, n=20, bounds=(-1, 1))

In [ ]:
for x in train_datasets[0]:
    print(x[0])
    break

In [ ]:
## Instantiate model
model = Model(
    encoding=encoding,
    input_size=2,
    output_size=2,
    hidden_dim=100,
    n_layers=1,
    device=device,
    init_std=0.03,
    # output_noise=0.01,
)

## TODO: Try starting with large weights

In [ ]:
import scipy.spatial as spatial


def dir_to_closest(X: pd.DataFrame):
    """Compute vectors from the closest vector to each vector."""
    dist = spatial.distance_matrix(X, X)
    dist[dist == 0] = np.inf
    closest = np.argmin(dist, axis=1)
    closest_loc = X.to_numpy()[closest]
    closest_vecs = X - closest_loc
    return closest_vecs


def track_function(inputs):
    # hidden_act = model(inputs)[0].detach().numpy()
    # hidden = pd.DataFrame(hidden_act, index=labels)
    # d = dir_to_closest(hidden)  # Not normalized
    grad = jacobian(lambda x: model(x)[0], inputs)
    # print(grad.shape)
    # print("Expect only n_datapoints, output_dim, input_dim")
    # Need to think about with respect to what we take derivative of.
    # print(np.linalg.det(grad))

    # theta = np.tensordot(grad, d, axes=(1, 1))
    # print(theta.shape)

    dets = torch.Tensor(
        [
            np.sum(
                [
                    torch.det(grad[input, :, input, char, :])
                    for char in range(grad.shape[3])
                ]
            )
            for input in range(grad.shape[0])
        ]
    )
    return dets


class WeightChangeTracker(Tracker):
    """Tracks how much weights have changed in the last epoch."""

    def __init__(self, model):
        self.last_weights = None
        self.model = model
        super().__init__()

    def track(self, *args):
        new_weights = [
            torch.Tensor(params[1]).clone() for params in self.model.named_parameters()
        ]

        change = 0.0
        if self.last_weights is not None:
            old_weights = self.last_weights
            for parameter_new, parameter_old in zip(new_weights, old_weights):
                change += float(torch.norm(parameter_new - parameter_old))
        self._trace.append(pd.DataFrame([change], ["Weight change"]))

        self.last_weights = new_weights

In [ ]:
## Setup compiler

# Define hyperparameters
n_epochs = 400
lr = 0.003

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
compiler = Compiler(model, criterion, optimizer)
initial_hidden = model.init_hidden(batch_size=1)[-1]
compiler.trackers = {
    "loss": ScalarTracker(lambda: compiler.validation(tracked_datasets)),
    "hidden": ActivationTracker(
        model, lambda inputs: model(inputs)[1][-1], initial=lambda: initial_hidden
    ),
    "output": ActivationTracker(model, lambda inputs: model(inputs)[0]),
    # "output_grad": ActivationTracker(
    #     model, lambda inputs: jacobian(lambda x: model(x)[0], inputs)
    # ),
    # "theta": ActivationTracker(model, track_function),
    "weight change": WeightChangeTracker(model),
}

In [ ]:
## Training run
compiler.training_run(
    train_datasets, tracked_datasets, n_epochs=n_epochs, batch_size=1000
)

In [ ]:
## Visualize automaton dynamics
data_hid = compiler.trackers["hidden"].get_trace()
data_output = compiler.trackers["output"].get_trace()
query = "Epoch >= 0"
query += "and (Dataset != 0)"
query += "and (Dataset <= 5)"
data_hid = data_hid.query(query).copy()
data_output = data_output.query(query).copy()

std = float(np.linalg.norm(data_hid.std()))

automaton_history = to_automaton_history(
    data_hid, data_output, merge_distance=0.05 * std
)

loss = compiler.trackers["loss"].get_trace()
val_loss = loss.query("Dataset==0")[0].to_numpy()
train_loss = loss.query("Dataset>0").groupby("Epoch").mean()

# mergers = np.array(automaton_history.get_state_changes()[0])
# splits = np.array(automaton_history.get_state_changes()[1])
n_states = np.array([len(automaton.states) for automaton in automaton_history])
# n_state_changes = np.abs(n_states[1:] - n_states[:-1])
# n_state_changes = scipy.signal.savgol_filter(n_state_changes, 31, 2)
# n_state_changes[n_state_changes < 0] = 0

# dets = compiler.trackers["theta"].get_trace()
# theta = dets.groupby("Epoch").sum()[0].to_numpy()
weight_change = compiler.trackers["weight change"].get_trace().to_numpy().reshape(-1)

animation = SliderAnimation(
    [
        ActivationsAnimation(data_hid, transform="PCA", n_labels=0),
        ActivationsAnimation(
            data_output, transform="none", n_labels=0, fixed_points=encoding.encoding
        ),
        AutomatonAnimation(automaton_history),
        EpochAnimation(
            graphs={
                "Training loss": train_loss,
                "Validation loss": val_loss,
            },
            unitless_graphs={
                "Number of states": n_states,
                # "average distance": av_distances,
                # "theta": theta,
                # "weight change": weight_change,
                # "state changes": n_state_changes,
            },
            # x_bounds=(0, 800),
            y_bounds=(0, 1),
        ),
    ],
    parameters=list(set(data_hid.index.get_level_values("Epoch"))),
    parameter_name="Epoch",
    fig_size=4,
)


# TODO Plot map A(h)
# TODO Plot pairwise distances
# TODO Plot total theta

In [ ]:
animation = SliderAnimation(
    [
        ActivationsAnimation(
            data_output, transform="none", n_labels=0, fixed_points=encoding.encoding
        ),
    ],
    parameters=list(set(data_hid.index.get_level_values("Epoch"))),
    parameter_name="Epoch",
    fig_size=4,
)

In [ ]:
# animation.to_gif("animation", step_size=int(len(train_loss) / 1000))

In [ ]:
## Compute d

data_hid = compiler.trackers["hidden"].get_trace()
hid = data_hid.loc[0]


av_distances = []
for epoch in range(n_epochs):
    vecs = dir_to_closest(data_hid.loc[epoch])
    av_distance = np.mean([np.sqrt(np.dot(vec, vec)) for vec in vecs.to_numpy()])
    av_distances.append(av_distance)


In [ ]:
## test

import torch.utils.data as data

dataset = train_datasets[5]
trainloader = data.DataLoader(dataset, batch_size=len(dataset), shuffle=False)
for batch in trainloader:
    inputs, outputs = batch

    # Get labels
    labels = []
    for input in inputs:
        try:
            decoding = model.encoding.decode(input.cpu())
            label = "".join(str(int(char)) for char in decoding)
        except KeyError:
            label = tuple(np.squeeze(input.cpu()).numpy())
        labels.append(label)


grad = jacobian(lambda x: model(x)[0], inputs)

dets = torch.Tensor(
    [
        np.sum(
            [
                torch.abs(torch.det(grad[input, :, input, char, :]))
                for char in range(grad.shape[3])
            ]
        )
        for input in range(grad.shape[0])
    ]
)

act_this_dataset = dets
act_this_dataset = torch.squeeze(act_this_dataset).cpu().detach().numpy()
act_this_dataset = pd.DataFrame(act_this_dataset, labels)
act_this_dataset


In [ ]:
## Check for generalization
N = 100
val_data = []
for n in trange(1, N + 1):
    val_data.append(seq_data(device, problem, encoding, n_datapoints=100, seq_len=n))
val_err = compiler.validation(val_data).to_numpy()[:, 0]

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.bar(np.arange(1, N + 1), val_err)
ax.set_xlabel("Sequence lengths")
ax.set_ylabel("Validation error")
ax.set_yticks(np.arange(0, 1, 0.1))
plt.show()


In [ ]:
## Make predictions


def predict(model, sequence):
    input = torch.from_numpy(np.array([encoding(sequence)], dtype=np.float32)).to(
        device
    )
    out, hidden = model(input)
    prediction = out.cpu().detach().numpy()
    return prediction


test_seq = [0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
print(f"Prediction: {predict(model, test_seq)}, True output: {problem(test_seq)}")
